In [1]:
# basic ipython configuration (reload source code automatically and plots inline)
%load_ext autoreload
%autoreload 2
%matplotlib inline

# standard python imports
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

from scipy.io import savemat
from scipy.io import loadmat

# tidy3D import
import tidy3d as td
from tidy3d import web

Using Tidy3D credentials from stored file


In [3]:
# 1 nanometer in units of microns (for conversion)
nm = 1e-3

wavelength = 600 * nm

# structure parameters
H = 320 * nm # element height [h will vary in the range (280, 360)]
L = 160 * nm # element width [width will varry in the range (120, 200)]
S = 480 * nm # unit-cell size
thickness_sub = 100 * nm # substrate thickness

space_pml = 1 * wavelength

# Number of unit cells in each x and y direction (NxN grid)
N = 120

# Define material properties at 600 nm
n_TiO2 = 2.6
n_SiO2 = 1.46
air = td.Medium(epsilon=1.0)
SiO2 = td.Medium(epsilon=n_SiO2**2)
TiO2 = td.Medium(epsilon=n_TiO2**2)

# resolution control
grids_per_wavelength = 30

# Number of PML layers to use along z direction
npml = 10

In [4]:
# grid size (um)
dl = wavelength / grids_per_wavelength

# using the wavelength in microns, one can use td.C_0 (um/s) to get frequency in Hz
# wavelength_meters = wavelength * meters
f0 = td.C_0 / wavelength

# Define PML layers, for this we have no PML in x, y but `npml` cells in z
pml_layers = [0, 0, npml]

# Compute the domain size in x, y (note: round down from side_length)
length_x = N * S
length_y = N * S
length_z = 2*space_pml + thickness_sub + 1.5*H

# construct simulation size array
sim_size = np.array([length_x, length_y, length_z])

# define substrate
substrate = td.Box(
    center=[0, 0, -length_z/2 + space_pml + thickness_sub / 2.0],
    size=[td.inf, td.inf, thickness_sub],
    material=SiO2)

# define coordinates of each unit cell
centers_x = S * np.arange(N) - length_x / 2.0 + S / 2.0
centers_y = S * np.arange(N) - length_y / 2.0 + S / 2.0
center_z = -length_z/2 + space_pml + thickness_sub

#############################################################################
# Bandwidth in Hz
fwidth = f0 / 10.0

# time dependence of source
gaussian = td.GaussianPulse(f0, fwidth, phase=0)

source = td.PlaneWave(
    source_time=gaussian,
    injection_axis='+z',
    position=-length_z/2 + space_pml / 10.0, # just next to PML
    polarization='y')

# Simulation run time past the source decay (around t=2*offset/fwidth)
run_time = 40 / fwidth

#############################################################################
# measure the field lambda/2 over the scatterers
monitor_xy = td.FreqMonitor(
    center=[0., 0., center_z + H + wavelength/2.0],
    size=[N*S, N*S, 0],
    freqs=[f0],
    store=['E'],
    name='xy_plane')

monitors=[monitor_xy]

#############################################################################
def box_maker(x_, y_, L_, H_):
    cent_ = np.array([x_, y_, center_z + H_ / 2.0])
    return td.Box(cent_, np.array([L_,L_, H_]), material=TiO2)

In [5]:
geometry = [substrate]
sim = td.Simulation(
    size=sim_size,
    mesh_step=[dl, dl, dl],
    structures=geometry,
    sources=[source],
    monitors=monitors,
    run_time=run_time,
    pml_layers=pml_layers)
# get the permittivity
eps_, _ = sim.epsilon(center=[0,0,0],size=[length_x,length_y,length_z]) 

Initializing simulation...
Mesh step (micron): [2.00e-02, 2.00e-02, 2.00e-02].
Simulation domain in number of grid points: [2880, 2880, 109].
Total number of computational grid points: 9.04e+08.
Total number of time steps: 23095.
Estimated data size (GB) of monitor xy_plane: 0.1991.


In [ ]:

E_lst = []
R_lst = []
H_lst = []
D_lst = []

for send_it in range(250):

    geometry = [substrate]
    
    # structure parameters
    rad_mat = np.random.choice([6.0, 8.0, 10.0, 12.0], (N,N))*dl
    rad_mat[rad_mat>10.0*dl] = 0
    hts_mat = np.random.choice([14.0, 16.0, 18.0], (N,N))*dl
    hts_mat[rad_mat==0]=0
    dsp_mat = np.random.choice([-2.0, 0.0, 2.0], (2,N,N))*dl
    dsp_mat[:,rad_mat==0] = 0


    for i, x in enumerate(centers_x):
        for j, y in enumerate(centers_y):
            if rad_mat[i,j]!=0:
                geometry.append(box_maker(x+dsp_mat[0,i,j], y+dsp_mat[1,i,j], rad_mat[i,j], hts_mat[i,j]))
                #geometry.append(box_maker(x+dsp_mat[0,i,j], y+dsp_mat[1,i,j], rad_mat[i,j], H))
                
    sim = td.Simulation(
        size=sim_size,
        mesh_step=[dl, dl, dl],
        structures=geometry,
        sources=[source],
        monitors=monitors,
        run_time=run_time,
        pml_layers=pml_layers)
    
    tsk_name = 'dta3d_p'+str(send_it)
    fldr_name = 'out3d_p'+str(send_it)
    
    project = web.new_project(sim.export(), task_name=tsk_name)
    web.monitor_project(project['taskId'])
    
    print('Downloading results: ', send_it)
    web.download_results(project['taskId'], target_folder=fldr_name)
    sim.load_results(fldr_name+'/monitor_data.hdf5')
    
    # get the field
    data = sim.data(monitors[0])
    E = data['E'].squeeze()
    E_lst.append(E)
    R_lst.append(rad_mat)
    H_lst.append(hts_mat)
    D_lst.append(dsp_mat)

In [ ]:
geometry = [substrate]
sim = td.Simulation(
    size=sim_size,
    mesh_step=[dl, dl, dl],
    structures=geometry,
    sources=[source],
    monitors=monitors,
    run_time=run_time,
    pml_layers=pml_layers)

tsk_name = 'dta3d_m'
fldr_name = 'out3d_m'

project = web.new_project(sim.export(), task_name=tsk_name)
web.monitor_project(project['taskId'])

print('Downloading results: ', send_it)
web.download_results(project['taskId'], target_folder=fldr_name)
sim.load_results(fldr_name+'/monitor_data.hdf5')

data = sim.data(monitors[0])
Em = data['E'].squeeze()

In [ ]:
for f_id in range(221,221+len(E_lst)):
    txt = './data3d_'+ str(f_id) +'.mat'
    savemat(txt,{'E':E_lst[f_id-221]-Em, 'R': R_lst[f_id-221]/dl, 'H': H_lst[f_id-221]/dl, 'D': D_lst[f_id-221]/dl})